**MOUNTING DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



WEIGHT AND BIASES INITIALS


In [ ]:
!pip install --upgrade wandb
import wandb
wandb.login()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

DEVICE ASSIGNMENT

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


IMPORT **LIBRARIES**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as t
from torchvision.datasets import MNIST
from torchsummary import summary
from torch.optim import SGD

In [ ]:
path = '/content/drive/MyDrive/Deep Learning/mnist'


In [ ]:
training_dataset = MNIST(root=path, train=True, transform=t.ToTensor(), download=True)
all_images = training_dataset.data


In [ ]:
all_images.shape

torch.Size([60000, 28, 28])

In [ ]:
all_images.view(3, -1).float().mean(dim=1)
all_images.view(3, -1).float().std(dim=1)

tensor([78.8755, 78.6755, 78.1487])

In [ ]:
raw_training_dataset = MNIST(root = path , train=True,
                         transform= t.Compose([t.ToTensor(),
                    t.Normalize((0.1307,), (0.3081,))]
), download=False, )
raw_testing_dataset = MNIST(root= path , train=False, transform=t.ToTensor(), download=False)


In [ ]:
print(len(raw_training_dataset))
print(len(raw_testing_dataset))

60000
10000


In [ ]:
training_ds, validation_ds = random_split(raw_training_dataset,[50000,10000])

In [ ]:
BATCH_SIZE = 128

In [ ]:
train_dl = DataLoader(training_ds, BATCH_SIZE, shuffle = True)
val_dl = DataLoader(validation_ds, BATCH_SIZE, shuffle = False)
test_dl = DataLoader(raw_testing_dataset, BATCH_SIZE, shuffle = False)


In [ ]:
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(6)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(120)


        self.fc1 = nn.Linear(120, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)


    def forward(self, X):
        out = self.conv1(X)
        out = self.bn1(out)
        out = F.relu(out)
        out = nn.AvgPool2d(kernel_size=2, stride=2, padding=1)(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out)
        out = nn.AvgPool2d(kernel_size=2, stride=2, padding=1)(out)

        out = self.conv3(out)
        out = self.bn3(out)
        out = F.relu(out)
        out = nn.AvgPool2d(kernel_size=2, stride=2, padding=1)(out)

        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)

        return out


 HYPERPARAMETERS

In [ ]:
LEARNING_RATE = 0.001
INPUT_SIZE = 784
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 20

In [ ]:
model = MNISTModel().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
       BatchNorm2d-2            [-1, 6, 24, 24]              12
            Conv2d-3             [-1, 16, 9, 9]           2,416
       BatchNorm2d-4             [-1, 16, 9, 9]              32
            Conv2d-5            [-1, 120, 1, 1]          48,120
       BatchNorm2d-6            [-1, 120, 1, 1]             240
            Linear-7                  [-1, 120]          14,520
            Linear-8                   [-1, 84]          10,164
            Linear-9                   [-1, 10]             850
Total params: 76,510
Trainable params: 76,510
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.08
Params size (MB): 0.29
Estimated Total Size (MB): 0.37
---------------------------------------------

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
run = wandb.init(project="mnist_lenet", config={
    "learning_rate": LEARNING_RATE,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
})

print(f"W&B Run URL: {run.url}")


wandb: Currently logged in as: oluwabusolaadubuola2023 (oluwabusolaadubuola2023-prometheus). Use `wandb login --relogin` to force relogin


W&B Run URL: https://wandb.ai/oluwabusolaadubuola2023-prometheus/mnist_lenet/runs/m6pz4ygm


In [ ]:
def train(model, train_dl, val_dl, epochs, criterion, optimizer, device):
    wandb.watch(model, log="all")

    for epoch in range(EPOCHS):
        model.train()
        epoch_train_loss = 0
        correct_train = 0
        total_train = 0



        for input, targets in train_dl:
            input, targets = input.to(device), targets.to(device)
            optimizer.zero_grad()
            pred = model(input)
            loss = criterion(pred, targets)
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()


            _, predicted = torch.max(pred, 1)
            correct_train += (predicted == targets).sum().item()
            total_train += targets.size(0)


        avg_train_loss = epoch_train_loss / len(train_dl)
        train_accuracy = 100 * correct_train / total_train



        model.eval()
        epoch_val_loss = 0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for input, targets in val_dl:
                input, targets = input.to(device), targets.to(device)
                pred = model(input)
                loss = criterion(pred, targets)
                epoch_val_loss += loss.item()

                _, predicted = torch.max(pred, 1)
                correct_val += (predicted == targets).sum().item()
                total_val += targets.size(0)

            avg_val_loss = epoch_val_loss / len(val_dl)
            val_accuracy = 100 * correct_val / total_val



        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
                  f'Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')



            wandb.log({
                "Train Loss": avg_train_loss,
                "Train Accuracy": train_accuracy,
                "Validation Loss": avg_val_loss,
                "Validation Accuracy": val_accuracy
            })


In [ ]:
train(model, train_dl, val_dl, EPOCHS, criterion, optimizer, device)

Epoch 10/20, Train Loss: 0.0140, Train Acc: 99.50%, Val Loss: 0.0371, Val Acc: 98.98%
Epoch 20/20, Train Loss: 0.0067, Train Acc: 99.78%, Val Loss: 0.0411, Val Acc: 98.98%


In [ ]:
def test(model, test_dl, criterion, device):
    with torch.no_grad():
        model.eval()
        test_loss = 0
        correct = 0
        total = 0

        wandb.init()

        for image, labels in test_dl:
            image, labels = image.to(device), labels.to(device)
            pred = model(image)
            loss = criterion(pred, labels)
            test_loss += loss.item()

            _, predicted = torch.max(pred, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        avg_test_loss = test_loss / len(test_dl)
        test_accuracy = 100 * correct / total

        print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

        wandb.log({
            "Test Loss": avg_test_loss,
            "Test Accuracy": test_accuracy
        })

        wandb.finish()

    return avg_test_loss, test_accuracy


In [ ]:
test(model, test_dl, criterion, device)

Test Loss: 2.3056, Test Accuracy: 10.10%


Test Accuracy,▁
Test Loss,▁
Test Accuracy,10.1
Test Loss,2.30564


(2.30564349210715, 10.1)